### Install Dependencies

In [1]:
%pip install --q langchain-community
%pip install --q langchain
%pip install --q duckduckgo-search

# TODO: add the following packages to poetry
%pip install --q requests
%pip install --q beautifulsoup4

#################################
# Required for PaperSpace Gradient
%pip install --q typing-inspect==0.8.0 typing_extensions==4.5.0
%pip install --q pydantic==1.10.8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest 7.2.1 requires attrs>=19.2.0, but you have attrs 18.2.0 which is incompatible.
gradient 2.0.6 requires marshmallow<3.0, but you have marshmallow 3.21.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires marshmallow<3.0, but you have marshmallow 3.21.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel t

### Setup Ollama

In [6]:
# %> curl -fsSL https://ollama.com/install.sh | sh
# %> ollama serve
# %> ollama pull nomic-embed-text     ???? do we need a retriever???
# %> ollama pull mixtral:instruct     (gemma:7b-instruct | mistral:instruct)

!ollama list

NAME                   	ID          	SIZE  	MODIFIED      
mixtral:instruct       	7708c059a8bb	26 GB 	6 seconds ago	
nomic-embed-text:latest	0a109f422b47	274 MB	4 minutes ago	


### Accelerator Info

In [7]:
!nvidia-smi

Tue Feb 27 22:04:19 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:00:05.0 Off |                  Off |
| 30%   46C    P8    21W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Prompt

In [8]:
template = """{text}
-----------
Using the above text, answer in short the following question:
> {question}
-----------
if the question cannot be answered using the text, imply summarize the text. Include all
factual information, numbers, stats etc if available.
"""

In [9]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)

### Ingest Documents

In [10]:
import requests
from bs4 import BeautifulSoup

def scrape_url(url: str):
    try:
        # set a get request to webpage
        response = requests.get(url)

        # if the request response was successful
        if (response.status_code == 200):
            # parse the content of the request
            soup = BeautifulSoup(response.text, "html.parser")
            text_content = soup.get_text(separator=" ", strip=True)
            return text_content
        else:
            return f"Failed to retrieve page: {response.status_code}"
    except Exception as e:
        print(e)    

    return ""

In [11]:
url = "https://blog.langchain.dev/announcing-langsmith/"

content = scrape_url(url)[:10000]

### Setup Local LLM & Embedding Models

In [12]:
LLM_MODEL = "mixtral:instruct"  # ("gemma:7b-instruct" | "mistral:instruct")
EMBEDDING_MODEL = "nomic-embed-text"
TEMPERATURE = 0.9
ENABLE_TRACING = False
DOCUMENT_CHUNK_SIZE = 7500  # Gemma --> DOCUMENT_CHUNK_SIZE=5000
CHUNK_OVERLAP = 100

#### Load & Test Local LLM Model

#### Testing the Ollama model. No need to import separate llm model.

In [13]:
from langchain.llms import Ollama

_test_llm_model_ = Ollama(
    model=LLM_MODEL,
    temperature=TEMPERATURE,
)

_test_llm_model_("Who are you?")

/usr/local/lib/python3.9/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' I am a language model trained by the Mistral AI team. I generate text based on the input I receive, and my purpose is to provide useful, safe, and accurate information.'

### Web Search Agent

In [14]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper

NUM_RESULTS_PER_QUESTION = 3
ddg_search = DuckDuckGoSearchAPIWrapper()

async def web_search(query: str, num_results: int = NUM_RESULTS_PER_QUESTION):
    web_query_results = ddg_search.results(query, num_results)
    return [r["link"] for r in web_query_results]


In [17]:
await web_search("What is langsmith?")

['https://blog.langchain.dev/announcing-langsmith/',
 'https://blog.logrocket.com/langsmith-test-llms-ai-applications/',
 'https://dev.to/logankilpatrick/what-is-langsmith-and-why-should-i-care-as-a-developer-19k']

### Setup Chat Chain

In [68]:
import asyncio
from langchain_community.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

scrape_and_summarize_chain = (
    RunnablePassthrough.assign(text=lambda x: scrape_url(x["url"])[:10000])
    | prompt
    | ChatOllama(model=LLM_MODEL)
    | StrOutputParser()
)

web_search_chain = (
    RunnablePassthrough.assign(urls=lambda x: asyncio.run(web_search(x["question"])))
    | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]])
    | scrape_and_summarize_chain.map()
)


chat_response = web_search_chain.invoke({"question": "What is LangSmith?"})
chat_response

[' LangSmith is a unified platform for developing, debugging, testing, evaluating, and monitoring large language model (LLM) applications. It provides a framework for creating LLM-based applications, as well as tools for tracing, visualizing, and comparing the inputs and outputs of different chains and prompts. LangSmith also includes a collection of evaluation modules, which use heuristics and LLMs to evaluate the correctness of an answer. Additionally, LangSmith allows developers to monitor the system-level performance of their application, track model/chain performance, debug issues, and understand how users are interacting with their application. LangSmith is used by companies such as Klarna, BCG, DeepLearningAI, Mendable, Multi-On, and Quivr for various purposes related to LLM application development and maintenance.',
 ' LangSmith is a platform that allows developers to build and evaluate language models. It provides a development environment for creating LangChain components, wh

#### Test Web Search

In [78]:
import json

questions_generation = (
    "Write 3 google search queries to search online that form an objective "
    "opinion from the following question: {question}\n "
    "Return a python list in the following format: "
    "['query 1', 'query 2', 'query 3']\n "
    "Do not include anything else after the list."
)

SEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            questions_generation,
        )
    ]
)

search_qanda_chain = (
    SEARCH_PROMPT 
    | ChatOllama(model=LLM_MODEL) 
    | StrOutputParser()
    | eval
    | (lambda l: [{"question": q} for q in l])
)

chain = search_qanda_chain | web_search_chain.map()

chain.invoke({
    "question": "what is json?"
})

[[" JSON (JavaScript Object Notation) is a lightweight data interchange format that is easy for humans to read and write and easy for machines to parse and generate. It is a text format that is completely language independent but uses conventions that are familiar to programmers of the C-family of languages, including C, C++, C#, Java, JavaScript, Perl, Python, and many others. JSON is built on two structures:\n\n1. A collection of name/value pairs. In various languages, this is realized as an object, record, struct, dictionary, hash table, keyed list, or associative array.\n2. An ordered list of values. In most languages, this is realized as an array, vector, list, or sequence.\n\nJSON is a subset of the JavaScript Programming Language, Standard ECMA-262 3rd Edition - December 1999. JSON is a text format that is completely language independent but uses conventions that are familiar to programmers of the C-family of languages, including C, C++, C#, Java, JavaScript, Perl, Python, and m

In [73]:
print_and_continue(['define JSON', 'what is the structure of JSON', 'JSON data type uses in programming'])

['define JSON', 'what is the structure of JSON', 'JSON data type uses in programming']


[{'question': 'define JSON'},
 {'question': 'what is the structure of JSON'},
 {'question': 'JSON data type uses in programming'}]